# 🎨 Data Designer Tutorial: Seeding Synthetic Data Generation with an External Dataset

#### 📚 What you'll learn

In this notebook, we will demonstrate how to seed synthetic data generation in Data Designer with an external dataset.

If this is your first time using Data Designer, we recommend starting with the [first notebook](https://nvidia-nemo.github.io/DataDesigner/latest/notebooks/1-the-basics/) in this tutorial series.


### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    ChatCompletionInferenceParams,
    DataDesigner,
    DataDesignerConfigBuilder,
    ModelConfig,
    SeedConfig,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is the main object is responsible for managing the data generation process.

- When initialized without arguments, the [default model providers](https://nvidia-nemo.github.io/DataDesigner/latest/concepts/models/default-model-settings/) are used.


In [2]:
data_designer_client = DataDesigner()

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see the [model config](https://nvidia-nemo.github.io/DataDesigner/latest/concepts/models/default-model-settings/) docs for more details).

- By default, we use [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the model provider configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nemotron-3-nano-30b-a3b"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v2"

# This sets reasoning to False for the nemotron-nano-v2 model.
SYSTEM_PROMPT = "/no_think"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=ChatCompletionInferenceParams(
            temperature=0.5,
            top_p=1.0,
            max_tokens=1024,
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🏥 Prepare a seed dataset

- For this notebook, we'll create a synthetic dataset of patient notes.

- We will _seed_ the generation process with a [symptom-to-diagnosis dataset](https://huggingface.co/datasets/gretelai/symptom_to_diagnosis).

- We already have the dataset downloaded in the [data](../data) directory of this repository.

<br>

> 🌱 **Why use a seed dataset?**
>
> - Seed datasets let you steer the generation process by providing context that is specific to your use case.
>
> - Seed datasets are also an excellent way to inject real-world diversity into your synthetic data.
>
> - During generation, prompt templates can reference any of the seed dataset fields.


In [5]:
# Download sample dataset from Github
import urllib.request

url = "https://raw.githubusercontent.com/NVIDIA/GenerativeAIExamples/refs/heads/main/nemo/NeMo-Data-Designer/data/gretelai_symptom_to_diagnosis.csv"
local_filename, headers = urllib.request.urlretrieve(url, "gretelai_symptom_to_diagnosis.csv")

seed_dataset = SeedConfig(dataset=local_filename)

# Pass the reference to the config builder for use during generation.
config_builder.with_seed_dataset(seed_dataset)

DataDesignerConfigBuilder(
    seed_dataset: 'gretelai_symptom_to_diagnosis.csv'
    seed_dataset_columns: ['diagnosis', 'patient_summary']
)

## 🎨 Designing our synthetic patient notes dataset

- Here we use `add_column` with keyword arguments (rather than imported config objects).

- Generally, we recommend using concrete objects, but this is a convenient shorthand.

- **Note**: The prompt template can reference fields from our seed dataset:
  - `{{ diagnosis }}` - the medical diagnosis from the seed data
  - `{{ patient_summary }}` - the symptom description from the seed data


In [6]:
config_builder.add_column(
    name="patient_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
)

config_builder.add_column(
    name="doctor_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
)

config_builder.add_column(
    name="patient_id",
    column_type="sampler",
    sampler_type="uuid",
    params={
        "prefix": "PT-",
        "short_form": True,
        "uppercase": True,
    },
)

config_builder.add_column(
    name="first_name",
    column_type="expression",
    expr="{{ patient_sampler.first_name}}",
)

config_builder.add_column(
    name="last_name",
    column_type="expression",
    expr="{{ patient_sampler.last_name }}",
)


config_builder.add_column(
    name="dob",
    column_type="expression",
    expr="{{ patient_sampler.birth_date }}",
)

config_builder.add_column(
    name="symptom_onset_date",
    column_type="sampler",
    sampler_type="datetime",
    params={"start": "2024-01-01", "end": "2024-12-31"},
)

config_builder.add_column(
    name="date_of_visit",
    column_type="sampler",
    sampler_type="timedelta",
    params={"dt_min": 1, "dt_max": 30, "reference_column_name": "symptom_onset_date"},
)

config_builder.add_column(
    name="physician",
    column_type="expression",
    expr="Dr. {{ doctor_sampler.last_name }}",
)

config_builder.add_column(
    name="physician_notes",
    column_type="llm-text",
    prompt="""\
You are a primary-care physician who just had an appointment with {{ first_name }} {{ last_name }},
who has been struggling with symptoms from {{ diagnosis }} since {{ symptom_onset_date }}.
The date of today's visit is {{ date_of_visit }}.

{{ patient_summary }}

Write careful notes about your visit with {{ first_name }},
as Dr. {{ doctor_sampler.first_name }} {{ doctor_sampler.last_name }}.

Format the notes as a busy doctor might.
""",
    model_alias=MODEL_ALIAS,
    system_prompt=SYSTEM_PROMPT,
)

config_builder.validate()

[22:44:02] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    seed_dataset: 'gretelai_symptom_to_diagnosis.csv'
    seed_dataset_columns: ['diagnosis', 'patient_summary']
    sampler_columns: [
        "patient_sampler",
        "doctor_sampler",
        "patient_id",
        "symptom_onset_date",
        "date_of_visit"
    ]
    llm_text_columns: ['physician_notes']
    expression_columns: [
        "first_name",
        "last_name",
        "dob",
        "physician"
    ]
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [7]:
preview = data_designer_client.preview(config_builder, num_records=2)

[22:44:02] [INFO] 👀 Preview generation in progress


[22:44:02] [INFO] ✅ Validation passed


[22:44:02] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[22:44:02] [INFO] 🩺 Running health checks for models...


[22:44:02] [INFO]   |-- 👀 Checking 'nvidia/nemotron-3-nano-30b-a3b' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...


...

[22:48:27] [INFO]   |-- 🎲 column: 'symptom_onset_date'


[22:48:27] [INFO]   |-- 🎲 column: 'date_of_visit'


[22:48:27] [INFO]   |-- 🧩 column: 'physician'


[22:48:27] [INFO]   |-- 📝 column: 'physician_notes'


[22:48:27] [INFO] 🥳 Preview complete!


In [8]:
# Run this cell multiple times to cycle through the 2 preview records.
preview.display_sample_record()

                                                   Seed Columns                                                    
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name            ┃ Value                                                                                         ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis       │ cervical spondylosis                                                                          │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ patient_summary │ I've been having a lot of pain in my neck and back. I've also been having trouble with my     │
│                 │ balance and coordination. I've been coughing a lot and my limbs feel weak.                    │
└─────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name               ┃ Value                                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler    │ {                                                                                          │
│                    │     'uuid': 'af44d1a3-693e-46b1-9504-75c6ea99646e',                                        │
│                    │     'locale': 'en_US',                                                                     │
│                    │     'first_name': 'Joe',                                                                   │
│                    │     'last_name': 'Gonzalez',                                                               │
│                    │     'middle_name': None,                                                                   │
│                    │     'sex': 'Male',                                                                         │
│                    │     'street_number': '448',                                                                │
│                    │     'street_name': 'Kaufman Viaduct',                                                      │
│                    │     'city': 'Sullivanberg',                                                                │
│                    │     'state': 'Kansas',                                                                     │
│                    │     'postcode': '46413',                                                                   │
│                    │     'age': 67,                                                                             │
│                    │     'birth_date': '1958-08-22',                                                            │
│                    │     'country': 'Bosnia and Herzegovina',                                                   │
│                    │     'marital_status': 'divorced',                                                          │
│                    │     'education_level': 'associates',                                                       │
│                    │     'unit': '',                                                                            │
│                    │     'occupation': 'Community pharmacist',                                                  │
│                    │     'phone_number': '8802363217',

In [9]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes,physician_notes__reasoning_trace
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,{'uuid': 'af44d1a3-693e-46b1-9504-75c6ea99646e...,{'uuid': 'dd921832-55e9-422b-9c11-92d2a56da5e6...,PT-F33D76C2,2024-10-14,2024-11-09,Joe,Gonzalez,1958-08-22,Dr. Collier,**Patient:** Joe Gonzalez **DOB:** 03/12/1978 ...,We need to write notes as Dr. Jennifer Collier...


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [10]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1                               │ 12                              │ 50.0%                                       │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                          ┃               data type ┃                           number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis                            │                  string │                                     1 (100.0%) │
├──────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────────────┤
│ patient_summary                      │                  string │                                     1 (100.0%) │
└──────────────────────────────────────┴─────────────────────────┴────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                       1 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                       1 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                       1 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                       1 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                       1 (100.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                          

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [11]:
job_results = data_designer_client.create(config_builder, num_records=10)

[22:48:27] [INFO] 🎨 Creating Data Designer dataset


[22:48:27] [INFO] ✅ Validation passed


[22:48:27] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[22:48:27] [INFO] 📂 Dataset path '/home/runner/work/DataDesigner/DataDesigner/docs/notebook_source/artifacts/dataset' already exists. Dataset from this session
		     will be saved to '/home/runner/work/DataDesigner/DataDesigner/docs/notebook_source/artifacts/dataset_12-17-2025_224827' instead.


[22:48:27] [INFO] 🩺 Running health checks for models...


...

[22:56:09] [INFO]   |-- 🧩 column: 'dob'


[22:56:09] [INFO]   |-- 🎲 column: 'symptom_onset_date'


[22:56:09] [INFO]   |-- 🎲 column: 'date_of_visit'


[22:56:09] [INFO]   |-- 🧩 column: 'physician'


[22:56:09] [INFO]   |-- 📝 column: 'physician_notes'


In [12]:
# Load the generated dataset as a pandas DataFrame.
dataset = job_results.load_dataset()

dataset.head()

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes,physician_notes__reasoning_trace
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,"{'age': 112, 'bachelors_field': 'stem', 'birth...","{'age': 44, 'bachelors_field': 'stem_related',...",PT-D76DD265,2024-09-13,2024-10-10,Brian,Dixon,1913-09-27,Dr. Fox,**Patient:** Brian Dixon **DOB:** 04/12/1978 *...,"We need to produce notes as a busy doctor, Dr...."
1,impetigo,I have a rash on my face that is getting worse...,"{'age': 53, 'bachelors_field': 'stem_related',...","{'age': 60, 'bachelors_field': 'no_degree', 'b...",PT-871D211C,2024-06-19,2024-07-01,Regina,Bush,1972-01-31,Dr. Wagner,**Patient:** Regina Bush **DOB:** 03/12/1992 *...,We need to produce notes as a primary care phy...
2,urinary tract infection,I have been urinating blood. I sometimes feel ...,"{'age': 58, 'bachelors_field': 'no_degree', 'b...","{'age': 21, 'bachelors_field': 'stem', 'birth_...",PT-2872DC58,2024-11-05,2024-12-01,Steven,Hernandez,1967-07-03,Dr. Rodriguez,**Patient:** Steven Hernandez **DOB:** 04/12...,"We need to write notes as a busy doctor, prima..."
3,arthritis,I have been having trouble with my muscles and...,"{'age': 89, 'bachelors_field': 'education', 'b...","{'age': 114, 'bachelors_field': 'business', 'b...",PT-B8D71844,2024-12-27,2025-01-11,James,Tran,1936-03-13,Dr. Riggs,"**Sonia Riggs, MD – Primary Care** **Visit:*...","We need to write notes as Dr. Sonia Riggs, a p..."
4,dengue,I have been feeling really sick. My body hurts...,"{'age': 104, 'bachelors_field': 'stem', 'birth...","{'age': 27, 'bachelors_field': 'no_degree', 'b...",PT-BE8BDAE4,2024-11-06,2024-11-09,Kimberly,Johnson,1921-07-23,Dr. Richardson,**Patient:** Kimberly Johnson **DOB:** 03/12/1...,"We need to write notes as a busy doctor might,..."


In [13]:
# Load the analysis results into memory.
analysis = job_results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 8                               │ 12                              │ 80.0%                                       │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                          ┃               data type ┃                           number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis                            │                  string │                                      5 (62.5%) │
├──────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────────────┤
│ patient_summary                      │                  string │                                     8 (100.0%) │
└──────────────────────────────────────┴─────────────────────────┴────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                       8 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                       8 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                       8 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                       8 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                       8 (100.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                          

## ⏭️ Next Steps

Check out the following notebook to learn more about:

- [Providing images as context](https://nvidia-nemo.github.io/DataDesigner/latest/notebooks/4-providing-images-as-context/)
